# Data exploration with Python

## EXERCISE: Reading and accessing data

### Read the survey response data

The `csv` module supports reading and writing of files in comma-separated values (CSV) and similar formats. We use `DictReader` since the first row of our survey responses file is a header. This produces a list of dictionaries, one dictionary per 57 responses. The `pprint` command below prints the dictionary corresponding the the first response.

In [1]:
import csv
import pprint
data = list(csv.DictReader(open('02_ds_survey_responses_raw_20160301.csv')))
pprint.pprint(data[0])

FileNotFoundError: [Errno 2] No such file or directory: '02_ds_survey_responses_raw_20160301.csv'

### Let's define constants for dictionary keys

Before moving on, let's define constants for the keys of this dictionary that will make it a bit easier to use.

In [ ]:
TIMESTAMP = 'Timestamp'
BACKGROUND_INDUSTRY = 'What main industry have you worked in?'
BACKGROUND_YEARS_PROFESSIONAL = 'How many years professional experience do you have?'
BACKGROUND_YEARS_PROGRAMMING = 'How many years programming experience do you have?'
BACKGROUND_SKILLS = 'What key experience do you have?'
IMPORT_DATA_MANAGEMENT = 'Data management'
IMPORT_STATISTICS = 'Statistics'
IMPORT_VISUALISATION = 'Visualisation'
IMPORT_MACHINE_LEARNING = 'Machine learning'
IMPORT_SOFTWARE_ENGINEERING = 'Software engineering'
IMPORT_COMMUNICATION = 'Communication'
GOALS_DEFINITION = 'How would you define data science in one sentence?'
GOALS_SKILLS = 'What key skills do you want to learn?'
GOALS_ROLE = 'What kind of role would you like to go into?'
GOALS_INDUSTRY = 'What industry would you like to go into?'
IMPORT_AREAS = [
    IMPORT_DATA_MANAGEMENT,
    IMPORT_STATISTICS,
    IMPORT_VISUALISATION,
    IMPORT_MACHINE_LEARNING,
    IMPORT_SOFTWARE_ENGINEERING,
    IMPORT_COMMUNICATION
    ]


### Accessing data values

This allows us to access cells in a row using the column name as a key. For example, the following prints the number of years professional experience for the first respondent. Note that the csv module reads all values as strings.

In [ ]:
row = data[0] # Row 0 corresponds to first respondent since arrays are 0-indexed
print("response:", row[BACKGROUND_YEARS_PROFESSIONAL]) # years of professional experience
print("type:", type(row[BACKGROUND_YEARS_PROFESSIONAL])) # csv 
print("type:", type(float(row[BACKGROUND_YEARS_PROFESSIONAL]))) # convert to float

### TODO What is the third respondent's rating for communication?

In [ ]:
data[2][IMPORT_COMMUNICATION]

## *STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.*

## EXERCISE: Frequency distribution and mode

### Counting data

`Counter` from the `collections` module is useful for quickly calculating frequencies. 

In [ ]:
from collections import Counter
c = Counter([row[IMPORT_COMMUNICATION] for row in data])
print("Distribution of communication importance ratings:")
for k, v in sorted(c.items()):
    print('{}: {}'.format(k, v))

### TODO Calculate distribution of background and goal industries

In [ ]:
def print_distr(data, column_key):
    print(column_key.upper())
    c = Counter([row[column_key] for row in data])
    for k,v in sorted(c.items()):
        print('* {}: {}'.format(k, v))
print_distr(data, BACKGROUND_INDUSTRY)
print_distr(data, GOALS_INDUSTRY)

### Calculating the mode

We can also use `Counter` to calcualte the mode.

In [ ]:
def mode(data, column_key):
    c = Counter([row[column_key] for row in data])
    return c.most_common(1)[0][0]
print("Communication mode:", mode(data, IMPORT_COMMUNICATION))

### TODO Calculate the mode of background and goal industries

In [ ]:
print("Background industry mode:", mode(data, BACKGROUND_INDUSTRY))
print("Goals industry mode:", mode(data, GOALS_INDUSTRY))

## *STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.*

## EXERCISE: Calculating descriptive statistics

### Cleaning float data

Which columns contained ratio data? We need to convert these to numeric types. Let's define a function since we have two ratio variables. Here we replace values that can't be converted with NaN (not a number).

In [ ]:
import warnings
import numpy as np
DEFAULT_VALUE = np.nan
def iter_clean(data, column_key, convert_function, default_value):
    for row in data:
        old_value = row[column_key]
        new_value = default_value
        try:
            new_value = convert_function(old_value)
        except (ValueError, TypeError):
            warnings.warn('Replacing {} with {} in column {}'.format(
                row[column_key], new_value, column_key))
        row[column_key] = new_value
        yield row
data = list(iter_clean(data, BACKGROUND_YEARS_PROFESSIONAL, float, DEFAULT_VALUE))
data = list(iter_clean(data, BACKGROUND_YEARS_PROGRAMMING, float, DEFAULT_VALUE))

### Cleaning timestamp data

We may also want to conver timestamp values.

In [ ]:
from datetime import datetime
FMT = "%m/%d/%Y %H:%M:%S"
def str_to_time(s):
    return datetime.strptime(s, FMT)
data = list(iter_clean(data, TIMESTAMP, str_to_time, DEFAULT_VALUE))

### Statistics with `numpy`

Once the data is converted, we can calculate descriptive statistics. `numpy` includes routines for measures of centrality and dispersion. Below we calculate descriptive statistics for professional and programming experience.

Further detail: http://docs.scipy.org/doc/numpy/reference/routines.statistics.html

In [ ]:
import numpy as np
for column_key in [BACKGROUND_YEARS_PROFESSIONAL, BACKGROUND_YEARS_PROGRAMMING]:
    v = [row[column_key] for row in data] # grab values
    print(column_key.upper())
    print("* Min..Max: {}..{}".format(np.nanmin(v), np.nanmax(v)))
    print("* Range: {}".format(np.nanmax(v)-np.nanmin(v)))
    print("* Mean: {}".format(np.nanmean(v)))
    print("* Standard deviation: {}".format(np.nanstd(v)))
    print("* Median: {}".format(np.nanmedian(v)))
    q1 = np.nanpercentile(v, 25)
    print("* 25th percentile (Q1): {}".format(q1))
    q3 = np.nanpercentile(v, 75)
    print("* 75th percentile (Q3): {}".format(q3))
    iqr = q3-q1
    print("* IQR: {}".format(iqr))

### Binning and histograms

`numpy` also provides routines for binning and producing histograms from ratio data.

NOTE RuntimeWarning due to NaN values, which are then ignored in histogram.

In [ ]:
v = [row[BACKGROUND_YEARS_PROFESSIONAL] for row in data] # grab values
freqs, bins = np.histogram(v, bins=7, range=(0,35)) # calculate frequencies and bin start/end
for i, freq in enumerate(freqs):
    # Note that bins[i] <= bin_values < bins[i+1]
    bin_str = '[{}..{})'.format(int(bins[i]), int(bins[i+1]))
    print(bin_str, ':', freq)

### TODO Calculate histogram for programming experience

In [ ]:
v = [row[BACKGROUND_YEARS_PROGRAMMING] for row in data] # grab values
freqs, bins = np.histogram(v, bins=7, range=(0,35)) # calculate frequencies and bin start/end
for i, freq in enumerate(freqs):
    # Note that bins[i] <= bin_values < bins[i+1]
    bin_str = '[{}..{})'.format(int(bins[i]), int(bins[i+1]))
    print(bin_str, ':', freq)

### EXTRA Calculate histograms with bin size of 2

In [ ]:
def get_histogram(data, column_key, bin_size, minimum, maximum):
    v = [row[column_key] for row in data] # grab values
    num_bins = (maximum-minimum)/bin_size
    return np.histogram(v, bins=num_bins, range=(minimum,maximum))

def print_histogram(freqs, bins, column_key):
    print(column_key.upper())
    for i, freq in enumerate(freqs):
        bin_str = '[{}..{})'.format(int(bins[i]), int(bins[i+1]))
        print(bin_str, ':', freq)
        
prof_freqs, prof_bins = get_histogram(data, BACKGROUND_YEARS_PROFESSIONAL, 2, 0, 30)
print_histogram(prof_freqs, prof_bins, BACKGROUND_YEARS_PROFESSIONAL)
prog_freqs, prog_bins = get_histogram(data, BACKGROUND_YEARS_PROGRAMMING, 2, 0, 30)
print_histogram(prog_freqs, prog_bins, BACKGROUND_YEARS_PROGRAMMING)

## *STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.*

## EXERCISE: Visualisation with matplotlib

### Making a frequency polygon

`matplotlib` provides functionality for creating various plots. Let's start with a frequency polygon. Note the line `%matplotlib inline` in is important in Jupyter.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
x_values = range(len(data))
professional_experience = [row[BACKGROUND_YEARS_PROFESSIONAL] for row in data]
plt.plot(x_values, professional_experience, 'g-', label='Professional')
plt.title('Experience')
plt.ylabel('Number of responses')
plt.legend(loc=2)
plt.show()

### TODO Add programming experience to the plot

Hint: Copy code above and add extra plt.plot command

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
x_values = range(len(data))
professional_experience = [row[BACKGROUND_YEARS_PROFESSIONAL] for row in data]
plt.plot(x_values, professional_experience, 'g-', label='Professional')
programming_experience = [row[BACKGROUND_YEARS_PROGRAMMING] for row in data]
plt.plot(x_values, programming_experience, 'r-', label='Programming')
plt.title('Experience')
plt.ylabel('Number of responses')
plt.legend(loc=2)
plt.show()

### A real frequency polygon

The above illustrate how to build a line chart in matplotlib but is not really frequency polygon as the x-axis corresponds to individual answers rather than groups. A more useful frequency poloygon uses the histogram data from above.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
x_values = list(range(len(prof_freqs)))
x_labels = ['[{}..{})'.format(int(prof_bins[i]), int(prof_bins[i+1])) for i in range(len(prof_freqs))]
plt.plot(x_values, prof_freqs, 'g-', label='Professional')
plt.plot(x_values, prog_freqs, 'r-', label='Programming')
plt.xticks(x_values, x_labels, rotation='vertical')
plt.title('Experience')
plt.ylabel('Number of responses')
plt.legend(loc=1)
plt.show()

### Making a bar chart

Now let's make a bar chart of communication importance.

In [ ]:
from collections import OrderedDict
IMPORT_KEYS = ['1', '2', '3', '4', '5']
def make_importance_plot(data, column_key, title):
    c = Counter(row[column_key] for row in data)
    d = OrderedDict([(k,c[k]) if k in c else (k,0) for k in IMPORT_KEYS])
    # bars are by default width 0.8, so we'll add 0.1 to the left coordinates
    xs = [i+0.1 for i,_ in enumerate(IMPORT_KEYS)]
    plt.bar(xs, d.values())
    plt.ylabel('Number of responses')
    plt.axis([0,5,0,35])
    plt.title(title)
    plt.xticks([i + 0.5 for i, _ in enumerate(IMPORT_KEYS)], IMPORT_KEYS)
    plt.show()
for a in IMPORT_AREAS:
    title = 'Importance of {}'.format(a.lower())
    make_importance_plot(data, a, title)

### TODO Make bar charts of known and future industries

In [ ]:
def iter_values(data, column_key):
    for row in data:
        yield(row[column_key])

background_industries = set(iter_values(data, BACKGROUND_INDUSTRY))
goals_industries = set(iter_values(data, GOALS_INDUSTRY))
all_industries = sorted(background_industries.union(goals_industries))

def make_bar_chart(data, column_key, values, y_min, y_max):
    c = Counter(row[column_key] for row in data)
    d = OrderedDict([(k,c[k]) if k in c else (k,0) for k in values])
    # bars are by default width 0.8, so we'll add 0.1 to the left coordinates
    xs = [i+0.1 for i,_ in enumerate(values)]
    plt.bar(xs, d.values())
    plt.ylabel('Number of responses')
    plt.axis([0,len(values),y_min,y_max])
    plt.title(column_key)
    plt.xticks([i + 0.5 for i, _ in enumerate(values)], values, rotation='vertical')
    plt.show()

make_bar_chart(data, BACKGROUND_INDUSTRY, all_industries, 0, 25)
make_bar_chart(data, GOALS_INDUSTRY, all_industries, 0, 25)

### Making a histogram

Now let's use the `histogram` from `numpy` above to create a histograms of professional and programming experience.

In [ ]:
def iter_histogram(data, column_key):
    v = [row[column_key] for row in data] # grab values
    freqs, bins = np.histogram(v, bins=7, range=(0,35))
    for i, freq in enumerate(freqs):
        yield ('[{}..{})'.format(int(bins[i]), int(bins[i+1])), freq)
        
def make_histogram_plot(data, column_key, title):
    d = OrderedDict(iter_histogram(data, column_key))
    keys = list(d.keys())
    xs = [i+0.1 for i,_ in enumerate(keys)]
    plt.bar(xs, d.values())
    plt.ylabel('Number of responses')
    plt.axis([0,7,0,35])
    plt.title(title)
    plt.xticks([i + 0.5 for i, _ in enumerate(keys)], keys)
    plt.show()
    
make_histogram_plot(data, BACKGROUND_YEARS_PROFESSIONAL, 'Professional experience')
make_histogram_plot(data, BACKGROUND_YEARS_PROGRAMMING, 'Programming experience')

### Making a scatterplot

Finally, let's make a scatterplot to compare professional and programming experience.

In [ ]:
professional_experience = [row[BACKGROUND_YEARS_PROFESSIONAL] for row in data]
programming_experience = [row[BACKGROUND_YEARS_PROGRAMMING] for row in data]
plt.scatter(professional_experience, programming_experience)
plt.title('Professional vs programming experience')
plt.xlabel('Years of professional experience')
plt.ylabel('Years of programming experience')
plt.axis([-1,35,-1,35])
plt.show()

## *STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.*

## EXERCISE: Box plots and correlation

### Visualising distributions with box plots

Mean and standard deviation are not informative for skewed data. `boxplot` is is a good visualisation for viewing and comparing distributions. It also shows outliers, e.g., values greater than `Q3+1.5*IQR` or less than `Q1-1.5*IQR`.

In [ ]:
professional_experience = [v for v in professional_experience if v is not np.nan]
programming_experience = [v for v in programming_experience if v is not np.nan]
plt.boxplot([professional_experience, programming_experience], 
            vert=False, notch=True, flierprops={'marker':'.'})
plt.axis([-1,35,0,3])
plt.yticks([1,2], ['Professional', 'Programming'])
plt.xlabel('Years experience')
plt.title('Professional vs programming experience')
plt.show()

### Calculating correlation between two variables

Pearson's r is the covariance of the two variables divided by the product of their standard deviations. Spearman rho is a common nonparametric test that is used in stead of Pearson's r when 

In [ ]:
from scipy import stats
# only keep rows where both professional and programming experience are defined
prof, prog = [], []
for row in data:
    if row[BACKGROUND_YEARS_PROFESSIONAL] is np.nan:
        continue # ignore rows with no value for professional experience
    elif row[BACKGROUND_YEARS_PROGRAMMING] is np.nan:
        continue # ignore rows with no value for programming experience
    else:
        prof.append(row[BACKGROUND_YEARS_PROFESSIONAL])
        prog.append(row[BACKGROUND_YEARS_PROGRAMMING])
print("Pearson (r, p): {}".format(stats.pearsonr(prof, prog)))

print(stats.spearmanr(prof, prog))

### TODO Calculate Kendall's tau between importance ratings

In [ ]:
print('{:.2f}'.format(3.14159))
for i,k1 in enumerate(IMPORT_AREAS):
    print(k1)
    v1 = [r[k1] for r in data]
    for j in range(i+1,len(IMPORT_AREAS),1):
        k2 = IMPORT_AREAS[j]
        v2 = [r[k2] for r in data]
        tau = stats.kendalltau(v1, v2)
        print('* {:5.2f} (p={:.3f}): {}'.format(tau.correlation, tau.pvalue, k2))

## *STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.*

## EXERCISE: Text data

### Simple tokenisation and word counts

Tokenisation is the process of breaking text into it's component parts, e.g., sentences, words. Below is a simple whitespace tokeniser that also removes some leading/trailing punctuation. We can use this to count the frequency of terms acros our data science definitions. 

In [ ]:
def tokenise(text):
    for word in text.lower().split():
        yield word.strip('.,')

def is_valid_word(w):
    if w == '':
        return False
    else:
        return True

def iter_ds_def_words(d):
    for row in d:
        for word in tokenise(row[GOALS_DEFINITION]):
            if is_valid_word(word):
                yield word

from collections import Counter
c = Counter(iter_ds_def_words(data))
print(c)

### Removing stop words

Very common function words can be removed to focus our analysis on content words.

In [ ]:
STOP_WORDS = frozenset([ # http://www.nltk.org/book/ch02.html#stopwords_index_term
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
    'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
    'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
    'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
    'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
    'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
    'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
    'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
    'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
    'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
    'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now'
    ])
def is_valid_word(w):
    if w == '':
        return False
    if w.lower() in STOP_WORDS:
        return False
    else:
        return True

c = Counter(iter_ds_def_words(data))
c

### Plotting term frequencies

Now we can build a simple horizontal bar chart that displays the most common terms across data science definitions.

In [ ]:
import operator

def iter_word_freqs(d, min_freq=4):
    c = Counter(iter_ds_def_words(d))
    for term, freq in c.items():
        if freq >= min_freq:
            yield term, freq

d = OrderedDict([(k,v) for k,v in sorted(iter_word_freqs(data), key=operator.itemgetter(1))])
ys = [i+0.5 for i,_ in enumerate(d)]
plt.barh(ys, d.values(), align='center')
plt.yticks(ys, list(d.keys()))
plt.axis([0,50,0-0.1,len(d)+0.1])
plt.show()